# Работа с форматом yml - 
### описание продуктов для market.yandex.ru

Пример описания (книги)

    <offer id="12342" type="book" available="true" bid="80">
      <url>http://best.seller.ru/product_page.asp?pid=14345</url>
      <price>80</price>
      <oldprice>90</oldprice>
      <currencyId>RUR</currencyId>
      <categoryId>3</categoryId>
      <picture>http://best.seller.ru/product_page.asp?pid=14345.jpg</picture>
      <store>false</store>
      <pickup>false</pickup>
      <delivery>true</delivery>
      <delivery-options>
        <option cost="200" days="1"/>
      </delivery-options>
      <author>Александра Маринина</author>
      <name>Все не так. В 2 томах. Том 1</name>
      <publisher>Эксмо</publisher>
      <series>А. Маринина — королева детектива</series>
      <year>2007</year>
      <ISBN>978-5-699-23647-3</ISBN>
      <volume>2</volume>
      <part>1</part>
      <language>rus</language>
      <binding>70x90/32</binding>
      <page_extent>288</page_extent>
      <description>
        Все прекрасно в большом патриархальном семействе Руденко...
      </description>
      <downloadable>false</downloadable>
      <age unit="year">18</age>
    </offer>
    
    
 Начало файла:
 
    <?xml version="1.0" encoding="windows-1251"?>
    <yml_catalog date="2016-02-05 17:22">
      <shop>
        <name>ABC</name>
        <company>ABC inc.</company>
        <url>http://www.abc.ru/</url>
        <currencies>
          <currency id="RUR" rate="1"/>
          <currency id="USD" rate="80"/>
        </currencies>
        <categories>
          <category id="1278">Электроника</category>
          <category id="3761" parentId="1278">Телевизоры</category>
          <category id="1553" parentId="3761">Медиа-плееры</category>
          <category id="3798">Бытовая техника</category>
          <category id="1293" parentId="3798">Холодильники</category>
        </categories>
        <delivery-options>
          <option cost="500" days="0" order-before="15"/>
          <option cost="300" days="1-3"/>
        </delivery-options>

In [147]:
from xml.etree import ElementTree as etree
from lxml import etree as lxml_etree
import csv

yml_stub = """
<yml_catalog date="{date}">
  <shop>
    <name>{name}</name>
    <company>{company}</company>
    <url>{url}</url>
    <currencies>
      <currency id="RUR" rate="1"/>
    </currencies>
    <categories>
    </categories>
    <delivery-options>
    </delivery-options>
    <offers>
    </offers>
  </shop>
</yml_catalog>
"""

def insert_categories(node, catList):
    for cat in catList:
        el = etree.SubElement(node, 'category')
        el.text = cat
        el.set('id', str(id(cat)))
        
def new_from_stub(data, stub):
    return etree.fromstring(stub.format(**data))

def write_tree(tree, fname='pricelist.yml'):
    tree.write(fname, encoding='utf-8', xml_declaration=True)
    tree = lxml_etree.parse(fname).write(encoding='utf-8', pretty_print=True)
    
def price_date():
    from datetime import datetime 
    return datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M")

def offer_from_dic(dic):
    attrs = ('id', 'type', 'available')
    el = etree.Element('offer', **{k:dic[k] for k in dic if k in attrs})
    for k in {k:dic[k] for k in dic if k not in attrs}:
        etree.SubElement(el, k).text = dic[k]
    return el

def insert_offers(offers_node, price):
    for item in price:
        offers_node.append(offer_from_dic(item))

def practica_book(book):
    for k in book:
        if k == 'category':
            yield 'categoryId', str(id(book[k]))
        elif k in ('delivery'):
            continue
        else:
            yield k, book[k]
    
def parse_practica_price(price_reader):
    return ({k: v for k, v in practica_book(book)} for book in price_reader)

def practica_categories(price_reader):
    return {book['category'] for book in price_reader}
        
def pretty_print(fname):
    

def main(price_fname='practica_price.csv'):
    data = dict(
        name="ИД Практика",
        company="ООО Издательский дом Практика",
        url="http://practica.ru",
        date=price_date(),
    )
    root = new_from_stub(data, yml_stub)
    shop = root.find('shop')
    categories_node = shop.find("categories")
    offers_node = shop.find("offers")
    
    with open(price_fname) as f:
        price = csv.DictReader(f, dialect='excel')
        insert_categories(categories_node, practica_categories(price))
        f.seek(0)
        price = csv.DictReader(f, dialect='excel')
        insert_offers(offers_node, parse_practica_price(price))
    
    tree = etree.ElementTree(root)
    write_tree(tree)

TypeError: write() takes exactly 1 positional argument (0 given)